In [1]:
import math
import io
import os
import torch
from torchvision import transforms
from torchvision.utils import save_image
import numpy as np
import time

from PIL import Image

In [2]:
from compressai.zoo import bmshj2018_factorized_relu
device = 'cuda' if torch.cuda.is_available() else 'cpu'
#device="cpu"
print(device)

cuda


In [4]:
quality=8
metric="mse" #"ms-ssim"
model_name="bmshj2018_factorized_relu"
net = bmshj2018_factorized_relu(quality=quality,metric=metric, pretrained=True).eval().to(device)

In [5]:
print(net)

FactorizedPrior(
  (entropy_bottleneck): EntropyBottleneck(
    (likelihood_lower_bound): LowerBound()
    (matrices): ParameterList(
        (0): Parameter containing: [torch.float32 of size 320x3x1 (cuda:0)]
        (1): Parameter containing: [torch.float32 of size 320x3x3 (cuda:0)]
        (2): Parameter containing: [torch.float32 of size 320x3x3 (cuda:0)]
        (3): Parameter containing: [torch.float32 of size 320x3x3 (cuda:0)]
        (4): Parameter containing: [torch.float32 of size 320x1x3 (cuda:0)]
    )
    (biases): ParameterList(
        (0): Parameter containing: [torch.float32 of size 320x3x1 (cuda:0)]
        (1): Parameter containing: [torch.float32 of size 320x3x1 (cuda:0)]
        (2): Parameter containing: [torch.float32 of size 320x3x1 (cuda:0)]
        (3): Parameter containing: [torch.float32 of size 320x3x1 (cuda:0)]
        (4): Parameter containing: [torch.float32 of size 320x1x1 (cuda:0)]
    )
    (factors): ParameterList(
        (0): Parameter containing: 

In [4]:
import torch
import onnx
from compressai.zoo import models
import onnxruntime

net = models["bmshj2018-factorized"](quality=6, metric="mse", pretrained=True)
# net = cheng2020_anchor(quality=5, pretrained=True).to(device)

# Some dummy input
x = torch.randn(1, 3, 1232, 1640, requires_grad=True)

export_path=r"C:\Swapnil\Narrowband_DRONE\Image_compression_code\Method_7_compress_ai\compressai\examples\bmshj2018_factorized_models"
export_path=os.path.join(export_path,"bmshj_model_GPU.onnx")
# Export the model
torch.onnx.export(net,                       # model being run
                  x,                         # model input (or a tuple for multiple inputs)
                  export_path,              # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=14,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['input'],   # the model's input names
                  output_names = ['output'], # the model's output names
                  dynamic_axes={'input': {0 : 'batch_size'},    # variable length axes
                                'output': {0 : 'batch_size'}}
                 )
print("Model is exported")

onnx_model = onnx.load(export_path)
onnx_model_graph = onnx_model.graph
onnx_session = onnxruntime.InferenceSession(onnx_model.SerializeToString())
# onnx_session = onnxruntime.InferenceSession("cheng2020.onnx")

input_shape = (1, 3, 1232, 1640)
x = torch.randn(input_shape).numpy()
print("size of x:",x.size)

input_names = ["input"]
output_names = ["output"]

onnx_output = onnx_session.run(output_names, {input_names[0]: x})[0]

C:\Swapnil\Narrowband_DRONE\Image_compression_code\Method_7_compress_ai\compressai\compressai\entropy_models\entropy_models.py:476: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  torch.tensor([1, 0], dtype=torch.long, device=x.device),
C:\Swapnil\Narrowband_DRONE\Image_compression_code\Method_7_compress_ai\compressai\compressai\entropy_models\entropy_models.py:488: TracerWarning: Iterating over a tensor might cause the trace to be incorrect. Passing a tensor of different shape won't change the number of iterations executed (and might lead to errors or silently give incorrect results).
  x = x.permute(*perm).contiguous()
C:\Swapnil\Narrowband_DRONE\Image_compression_code\Method_7_compress_ai\compressai\compressai\entropy_models\entro

Model is exported
size of x: 6061440


In [9]:
import cv2
import onnx
import numpy as np
import onnxruntime
from PIL import Image
import time

def preprocess_image(image_path):
    """
    Preprocess the input image to the format expected by the model.
    """
    # Load the image using PIL
    image = Image.open(image_path).convert("RGB")

    image=image.resize((1640,1232))
    
    # Convert to numpy array and normalize the image (0 to 1)
    image = np.array(image) / 255.0
    
    # Convert to CHW format (Channels x Height x Width)
    image = np.transpose(image, (2, 0, 1))
    
    # Add a batch dimension (1, Channels, Height, Width)
    image = np.expand_dims(image, axis=0).astype(np.float32)
    
    return image

def save_compressed_image(output, save_path):
    """
    Save the compressed output image from the model inference.
    """
    # Post-process the output (if needed)
    output = output.squeeze(0)  # Remove batch dimension
    
    # Convert to HWC format (Height x Width x Channels)
    output = np.transpose(output, (1, 2, 0))
    
    # Clip values to valid range (0, 1) and convert to 8-bit (0-255)
    output = np.clip(output, 0, 1) * 255.0
    output = output.astype(np.uint8)
    
    # Convert to image format and save
    output_image = Image.fromarray(output)
    output_image.save(save_path)

def run_inference(image_path, model_path, save_path):
    """
    Run inference on an input image using the exported ONNX model.
    """
    # Load the image and preprocess it
    input_image = preprocess_image(image_path)
    
    # Load the ONNX model

    session_options = onnxruntime.SessionOptions()
    session_options.graph_optimization_level = onnxruntime.GraphOptimizationLevel.ORT_ENABLE_ALL
    #session_options.log_severity_level = onnxruntime.logging.LoggingLevel.WARNING

    #Check if CUDA is available
    providers = ['CUDAExecutionProvider'] if 'CUDAExecutionProvider' in onnxruntime.get_available_providers() else ['CPUExecutionProvider']
    print("Using:",providers)
    onnx_session = onnxruntime.InferenceSession(model_path,sess_options=session_options, providers=providers)
    #onnx_session = onnxruntime.InferenceSession(model_path)

    logging.info(f"Using execution provider(s) :{onnx_session.get_providers()}")
    
    # Run inference
    input_names = ["input"]
    output_names = ["output"]
    start=time.time()
    onnx_output = onnx_session.run(output_names, {input_names[0]: input_image})[0]
    end=time.time()

    print("Time in seconds for the inference is:",round(end-start,2),"seconds")

    
    # Save the output image
    save_compressed_image(onnx_output, save_path)
    print(f"Compressed image saved at: {save_path}")
import logging
# Configure logging
#logging.basicConfig(level=logging.INFO, format='%(message)s')

image_path=r"C:\Swapnil\Narrowband_DRONE\Image_compression_code\FLOPS\image.png"
model_path=r"C:\Swapnil\Narrowband_DRONE\Image_compression_code\Method_7_compress_ai\compressai\examples\bmshj2018_factorized_models\bmshj_model_GPU.onnx"
save_path=r"C:\Swapnil\Narrowband_DRONE\Image_compression_code\FLOPS\image_py_onnx_export_GPU.png"

run_inference(image_path,model_path,save_path)
# Log a message
logging.info("ONNX Runtime session created")

Using execution provider(s) :['CPUExecutionProvider']


Using: ['CUDAExecutionProvider']
Time in seconds for the inference is: 4.96 seconds


ONNX Runtime session created


Compressed image saved at: C:\Swapnil\Narrowband_DRONE\Image_compression_code\FLOPS\image_py_onnx_export_GPU.png


In [5]:
import onnxruntime
print(onnxruntime.get_available_providers())

['TensorrtExecutionProvider', 'CUDAExecutionProvider', 'CPUExecutionProvider']


In [40]:
print(type(onnx_output),onnx_output.shape,onnx_output.size)

<class 'numpy.ndarray'> (1, 3, 1232, 1648) 6091008


In [62]:
#exporting the BMSHJ model to onnx
#torch_model=net
#torch_input=torch.randn(1,1,1280,720)

#options and method to run the inference from the ONNX model using the GPU

#session_options = onnxruntime.SessionOptions()
#session_options.graph_optimization_level = onnxruntime.GraphOptimizationLevel.ORT_ENABLE_ALL
#session = onnxruntime.InferenceSession(export_path,sess_options=session_options, providers=['CUDAExecutionProvider'])
#onnx_program=torch.onnx.dynamo_export(torch_model,torch_input)

In [53]:

#CNN model 
import torch
import torch.nn as nn
import torch.nn.functional as F


class MyModel(nn.Module):

    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


import onnx
import onnxruntime
torch_model=MyModel() #PyTorch model
torch_input=torch.randn(1,1,32,32) #input for the torch model
onnx_program=torch.onnx.dynamo_export(torch_model,torch_input) #export the pytorch model to onnx

onnx_program.save('my_image_classifier.onnx') #save the model to disc


onnx_model = onnx.load("my_image_classifier.onnx") #loading the onnx model
onnx.checker.check_model(onnx_model)#check the model


onnx_input = onnx_program.adapt_torch_inputs_to_onnx(torch_input)
print(f"Input length: {len(onnx_input)}")
print(f"Sample input: {onnx_input}")

ort_session = onnxruntime.InferenceSession("./my_image_classifier.onnx", providers=['CPUExecutionProvider'])

def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

onnxruntime_input = {k.name: to_numpy(v) for k, v in zip(ort_session.get_inputs(), onnx_input)}

onnxruntime_outputs = ort_session.run(None, onnxruntime_input)

torch_outputs = torch_model(torch_input)
torch_outputs = onnx_program.adapt_torch_outputs_to_onnx(torch_outputs)

assert len(torch_outputs) == len(onnxruntime_outputs)
for torch_output, onnxruntime_output in zip(torch_outputs, onnxruntime_outputs):
    torch.testing.assert_close(torch_output, torch.tensor(onnxruntime_output))

print("PyTorch and ONNX Runtime output matched!")
print(f"Output length: {len(onnxruntime_outputs)}")
print(f"Sample output: {onnxruntime_outputs}")

OnnxExporterError: Failed to export the model to ONNX. Generating SARIF report at 'report_dynamo_export.sarif'. SARIF is a standard format for the output of static analysis tools. SARIF logs can be loaded in VS Code SARIF viewer extension, or SARIF web viewer (https://microsoft.github.io/sarif-web-component/). Please report a bug on PyTorch Github: https://github.com/pytorch/pytorch/issues

PyTorch and ONNX Runtime output matched!
Output length: 1
Sample output: [array([[ 0.01380568, -0.0715049 , -0.02148017, -0.10276306,  0.00692059,
        -0.1345982 ,  0.04789107, -0.02346265, -0.13729787,  0.01007874]],
      dtype=float32)]
